In [1]:
import cv2
import nibabel as nib
import numpy as np
from pycimg import CImg
import matplotlib.pyplot as plt


In [2]:
IMG_NUM = 1

fname = './Images/IMG_%04d.nii.gz' % IMG_NUM
lungs = nib.load(fname).get_fdata()
fname = './BodyMasks/BODYMASK_IMG_0001.nii.gz'
bodymask_gt = nib.load(fname).get_fdata()
max_val = np.max(lungs)

lungs_bin_inv = np.zeros(lungs.T.shape)
for i, dim in enumerate(lungs.T):
    _, lungs_bin_inv[i] = cv2.threshold(dim, -320, max_val, cv2.THRESH_BINARY_INV)

lungs_bin_inv = lungs_bin_inv.T
# CImg(lungs_bin_inv).display();


In [48]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
bodymask_pred = np.zeros(lungs.T.shape)
for i, dim in enumerate(lungs.T):
    _, bin = cv2.threshold(dim, -200, max_val, cv2.THRESH_BINARY)
    eroded = cv2.erode(bin, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)
    # negated = np.logical_not(dilated)
    cv2.floodFill(dilated.astype(np.uint8), np.zeros((dilated.shape[0]+2, dilated.shape[1]+2), dtype=np.uint8), seedPoint=(0, 0), newVal=255)
    
    bodymask_pred[i] = dilated

bodymask_pred = bodymask_pred.T
CImg(bodymask_pred).display();

In [ ]:
lungs_air_within_body = np.logical_and(lungs_bin_inv, bodymask_gt).astype(np.uint8)
CImg(lungs_air_within_body).display();

In [3]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
lungs_morphed = np.zeros(lungs_air_within_body.T.shape)
for i, dim in enumerate(lungs_air_within_body.T):
    lungs_morphed[i] = cv2.medianBlur(dim, 7)
lungs_morphed = lungs_morphed.T

lungs_morphed = cv2.dilate(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.erode(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.erode(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.dilate(lungs_morphed, kernel, None, iterations=2)
CImg(lungs_morphed).display()


CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [4]:
lungs_processed = np.zeros(lungs_morphed.T.shape)
for i, dim in enumerate(lungs_morphed.T):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dim.astype(np.uint8))
    for j in range(num_labels):
        if stats[j][cv2.CC_STAT_AREA] < 500:
            labels[labels == j] = 0

    lungs_processed[i] = labels

print(num_labels)
for i, dim in enumerate(lungs_processed):
    _, lungs_processed[i] = cv2.threshold(dim, 0, 255, cv2.THRESH_BINARY)

lungs_processed = lungs_processed.T.astype(np.uint8)
CImg(lungs_processed).display()

1


CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [13]:
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage import filters

boundaries = np.zeros(lungs_processed.T.shape)
gradients = np.zeros(lungs_processed.T.shape)
sure_fg_3D = np.zeros(lungs_processed.T.shape)
for i, (processed_img, img) in enumerate(zip(lungs_processed.T, lungs.T)):
    gradients[i] = filters.sobel(processed_img)

    # distance = ndi.distance_transform_edt(processed_img)
    # _, sure_fg = cv2.threshold(distance, 0.6*distance.max(), 255, 0)
    # sure_fg = sure_fg.astype(np.uint8)
    sure_fg = cv2.erode(processed_img, np.ones((7, 7)), iterations=3).astype(np.uint8)
    sure_bg = cv2.dilate(processed_img, np.ones((7, 7)), iterations=3).astype(np.uint8)
    boundaries[i] = cv2.subtract(sure_bg, sure_fg)
    sure_fg_3D[i] = sure_fg

boundaries = boundaries.T
sure_fg_3D = sure_fg_3D.T
markers, num_labels = ndi.label(sure_fg_3D)
markers += 1
markers[boundaries == 255] = 0
labels_sums = np.zeros(num_labels)
for i in range(num_labels):
    labels_sums[i] = np.sum((markers==i)*1)

top_arg =  np.argmax(labels_sums)
print(top_arg)
trimmed_labels = np.zeros(markers.shape)
trimmed_labels += (markers == top_arg) * markers
for i in range(1, 4):
    labels_sums[top_arg] = 0
    top_arg = np.argmax(labels_sums)
    print(top_arg)
    trimmed_labels += (markers == top_arg) * markers

markers = trimmed_labels.astype(np.int32)
CImg(markers).display()

gradients /= np.max(gradients)
gradients *= 255
gradients = gradients.T.astype(np.int32)
CImg(gradients).display()


1
0
7
9


CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [14]:
lungs_done = watershed(gradients, markers)
CImg(lungs_done).display()

CImg(array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],

In [21]:
CImg(lungs_done).display()

CImg(array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],

In [22]:
fname = './ReferenceSegmentations/LUNGS_IMG_%04d.nii.gz' % IMG_NUM
lungs_test = nib.load(fname).get_fdata()
CImg(lungs_test).display()

CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [23]:
import surface_distance
from surface_distance import metrics

left_lung_pred = lungs_done == 9
right_lung_pred = lungs_done == 7

left_lung_gt = lungs_test == 3
right_lung_gt = lungs_test == 2

dists_ideal = surface_distance.compute_surface_distances(left_lung_gt, left_lung_gt, (1, 1, 1))
hausdorff_ideal = surface_distance.compute_robust_hausdorff(dists_ideal, 100)
dice_vol_coef_ideal = surface_distance.compute_dice_coefficient(left_lung_gt, left_lung_gt)

dists_left = surface_distance.compute_surface_distances(left_lung_gt, left_lung_pred, (1, 1, 1))
hausdorff_left = surface_distance.compute_robust_hausdorff(dists_left, 100)
dice_vol_coef_left = surface_distance.compute_dice_coefficient(left_lung_gt, left_lung_pred)

dists_right = surface_distance.compute_surface_distances(right_lung_gt, right_lung_pred, (1, 1, 1))
hausdorff_right = surface_distance.compute_robust_hausdorff(dists_right, 100)
dice_vol_coef_right = surface_distance.compute_dice_coefficient(right_lung_gt, right_lung_pred)

print(hausdorff_ideal, dice_vol_coef_ideal)
print(hausdorff_left, dice_vol_coef_left)
print(hausdorff_right, dice_vol_coef_right)


0.0 1.0
23.280893453645632 0.9618070608082716
23.895606290697042 0.9649420472641006
